In [1]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import LeaveOneOut, KFold

In [2]:
train_files_labels = np.loadtxt("./train.csv", dtype=str, delimiter=",").tolist()

In [3]:
files, labels = zip(*[(l[0], int(l[1])) for l in train_files_labels])

In [4]:
print(files[:3])
print(labels[:3])

('./mnist_train/3/mnist_train_37167.png', './mnist_train/4/mnist_train_4346.png', './mnist_train/6/mnist_train_46003.png')
(3, 4, 6)


In [9]:
for train_index, test_index in KFold(3).split(files):
    print(np.array(files)[train_index])
    print(np.array(files)[test_index])
    print(np.array(files)[train_index].shape)
    print(np.array(files)[test_index].shape)
    print("------------------------------------------------")

['./mnist_train/5/mnist_train_50019.png'
 './mnist_train/1/mnist_train_54675.png'
 './mnist_train/7/mnist_train_28781.png' ...
 './mnist_train/7/mnist_train_17533.png'
 './mnist_train/1/mnist_train_58991.png'
 './mnist_train/0/mnist_train_28101.png']
['./mnist_train/3/mnist_train_37167.png'
 './mnist_train/4/mnist_train_4346.png'
 './mnist_train/6/mnist_train_46003.png' ...
 './mnist_train/2/mnist_train_39739.png'
 './mnist_train/4/mnist_train_24810.png'
 './mnist_train/8/mnist_train_29729.png']
(32000,)
(16000,)
------------------------------------------------
['./mnist_train/3/mnist_train_37167.png'
 './mnist_train/4/mnist_train_4346.png'
 './mnist_train/6/mnist_train_46003.png' ...
 './mnist_train/7/mnist_train_17533.png'
 './mnist_train/1/mnist_train_58991.png'
 './mnist_train/0/mnist_train_28101.png']
['./mnist_train/5/mnist_train_50019.png'
 './mnist_train/1/mnist_train_54675.png'
 './mnist_train/7/mnist_train_28781.png' ...
 './mnist_train/0/mnist_train_1625.png'
 './mnist_train

In [31]:
# 从iterator创建数据集，好处是使用交叉验证进行处理数据
def make_dataset(x_data, y_data, n_splits):
    def gen():
        for train_index, test_index in KFold(n_splits).split(x_data):
            x_train, x_test = np.array(x_data)[train_index], np.array(x_data)[test_index]
            y_train, y_test = np.array(y_data)[train_index], np.array(y_data)[test_index]
            yield x_train, y_train, x_test, y_test
    return tf.data.Dataset.from_generator(gen, (tf.string, tf.int32, tf.string, tf.int32))

In [32]:
dataset = make_dataset(files, labels, 3)

In [33]:
# iterator = dataset.make_one_shot_iterator()
# next_element = iterator.get_next()

In [47]:
with tf.Session() as sess:
    try:
        while True:
            tmp = sess.run(next_element)
            print(len(tmp))
            print(len(tmp[0]))
            print(len(tmp[3]))
    except tf.errors.OutOfRangeError:
        pass

4
32000
16000
4
32000
16000
4
32000
16000


In [35]:
# dataset数据集转换函数
def _parse_image(filename, label):
    image_strings = tf.read_file(filename)
    images_decoded = tf.image.decode_png(image_strings)
    images_reshaped = tf.reshape(images_decoded, [28, 28, 1])
    images_converted = tf.cast(images_reshaped, tf.float32)
    images_scaled = tf.divide(images_converted, 255.)
    labels = tf.cast(label, tf.int32)
    return images_scaled, labels

In [40]:
dataset = dataset.batch(100)
print(dataset.output_types)
print(dataset.output_shapes)

(tf.string, tf.int32, tf.string, tf.int32)
(TensorShape(None), TensorShape(None), TensorShape(None), TensorShape(None))
